# Explore the Data

In [29]:
import pandas as pd
df = pd.read_csv('nonvoters_data.csv')

df.head()

,RespId,weight,Q1,Q2_1,Q2_2,Q2_3,Q2_4,Q2_5,Q2_6,Q2_7,...,Q30,Q31,Q32,Q33,ppage,educ,race,gender,income_cat,voter_category
0,470001,0.7516,1,1,1,2,4,1,4,2,...,2,NaN,1.0,NaN,73,College,White,Female,$75-125k,always
1,470002,1.0267,1,1,2,2,3,1,1,2,...,3,NaN,NaN,1.0,90,College,White,Female,$125k or more,always
2,470003,1.0844,1,1,1,2,2,1,1,2,...,2,NaN,2.0,NaN,53,College,White,Male,$125k or more,sporadic
3,470007,0.6817,1,1,1,1,3,1,1,1,...,2,NaN,1.0,NaN,58,Some college,Black,Female,$40-75k,sporadic
4,480008,0.9910,1,1,1,-1,1,1,1,1,...,1,-1.0,NaN,NaN,81,High school or less,White,Male,$40-75k,always


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5836 entries, 0 to 5835
Columns: 119 entries, RespId to voter_category
dtypes: float64(23), int64(91), object(5)
memory usage: 5.3+ MB


By looking at the info from our dataframe, we learn that we have 119 columns in our dataset and 5,836 rows. 

There's too many columns to see the full number and so, to find out information about the columns, [we refer to the codebook for the dataset](https://github.com/fivethirtyeight/data/blob/master/non-voters/nonvoters_codebook.pdf).

This analysis will focus on questions two and three in the study and so we'll filter the dataframe to only include the pertinent columns.


2. In your view, how important are each of the following to being a good American?

    1. Voting in elections
    2. Serving on a jury
    3. Following what happens in government and politics
    4. Displaying the American flag
    5. Participating in the U.S. Census every ten years
    6. Knowing the Pledge of Allegiance
    7. Supporting the military
    8. Respecting the opinions of those who disagree with you
    9. Believing in God
    10. Protesting if you believe government actions are wrong
        
Responses
1. Very important
2. Somewhat important
3. Not so important
4. Not at all important


3. How much do you agree or disagree with the following statements?

    1. Systemic racism is a problem in the United States.
    2. Systemic racism in policing is a bigger problem than violence and vandalism in protests.
    3. Society as a whole has become too soft and feminine.
    4. The mainstream media is more interested in making money than telling the truth.
    5. Traditional parties and politicians don’t care about people like me.
    6. The way people talk needs to change with the times to be more sensitive to people with different backgrounds.

Responses
1. Strongly agree
2. Somewhat agree
3. Somewhat disagree
4. Strongly disagree

In [31]:
df[["Q2_1","Q2_2","ppage"]].describe()


,Q2_1,Q2_2,ppage
count,5836.000000,5836.000000,5836.000000
mean,1.246402,1.705106,51.693797
std,0.660253,0.866346,17.071561
min,-1.000000,-1.000000,22.000000
25%,1.000000,1.000000,36.000000
50%,1.000000,2.000000,54.000000
75%,1.000000,2.000000,65.000000
max,4.000000,4.000000,94.000000


Now that we've inspected our data, we can make a list of the ways we'll need to restructure the data.

1) Create bins for our age data: create quartiles for age data
2) Filter dataframe for the specific questions: questions two and three 
3) Remove values of -1 
4) Likert Scale data needs to be relabeled
5) Rename Columns

# Clean and Restructure the Data

## Create bins for our age data: create quartiles for age data

In [32]:
# Use quartile values to create bins

age_bins = ["22-36","37-54","55-65","66-94"]

df['age_binned'] = pd.qcut(df['ppage'],
                              q=[0,0.25, 0.5, 0.75, 1],
                              labels=age_bins)

df['age_binned']


0       66-94
1       66-94
2       37-54
3       55-65
4       66-94
        ...  
5831    22-36
5832    55-65
5833    37-54
5834    22-36
5835    22-36
Name: age_binned, Length: 5836, dtype: category
Categories (4, object): ['22-36' < '37-54' < '55-65' < '66-94']

## Filter dataframe for the specific questions

In [33]:
import re

# Create new dataframe that only includes demographic and columns for questions two and three. 

# Create full column list of all columns in the DataFrame
full_column_list = list(df.columns)

# Create an empty list that we'll use to append the names of the pertinent question columns (for questions two and three)
question_columns = []

# Create an empty list that we'll use to append the name of all the demographic columns
demographic_columns = []

for column in full_column_list:
    # Append any columns that contain the text Q2_ or Q3_ (those for questions two and three)
    if re.search("Q2_.+", column) or re.search("Q3_.+", column):
        question_columns.append(column)
    # Append demographic criteria to the list of demographic variables
    elif df[column].dtype == object:
        demographic_columns.append(column)
    # Append binned data column
    elif re.search("age_binned",column):
        demographic_columns.append(column)

# Create full list of columns we'll use for our final DataFrame
full_column_list = (question_columns+demographic_columns)

# Create a copy of the dataframe we'll use for our final df
subset_df = df[full_column_list].copy()

subset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5836 entries, 0 to 5835
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Q2_1            5836 non-null   int64   
 1   Q2_2            5836 non-null   int64   
 2   Q2_3            5836 non-null   int64   
 3   Q2_4            5836 non-null   int64   
 4   Q2_5            5836 non-null   int64   
 5   Q2_6            5836 non-null   int64   
 6   Q2_7            5836 non-null   int64   
 7   Q2_8            5836 non-null   int64   
 8   Q2_9            5836 non-null   int64   
 9   Q2_10           5836 non-null   int64   
 10  Q3_1            5836 non-null   int64   
 11  Q3_2            5836 non-null   int64   
 12  Q3_3            5836 non-null   int64   
 13  Q3_4            5836 non-null   int64   
 14  Q3_5            5836 non-null   int64   
 15  Q3_6            5836 non-null   int64   
 16  educ            5836 non-null   object  
 17  race          

In [34]:
# Remove values that are below zero 

for name in question_columns:
    subset_df.drop(subset_df[subset_df[name]<0].index, inplace = True)

subset_df.describe()

,Q2_1,Q2_2,Q2_3,Q2_4,Q2_5,Q2_6,Q2_7,Q2_8,Q2_9,Q2_10,Q3_1,Q3_2,Q3_3,Q3_4,Q3_5,Q3_6
count,5570.000000,5570.000000,5570.000000,5570.000000,5570.000000,5570.000000,5570.000000,5570.000000,5570.000000,5570.000000,5570.000000,5570.000000,5570.00000,5570.000000,5570.000000,5570.000000
mean,1.256912,1.717056,1.652065,2.203232,1.286176,1.826032,1.511131,1.472711,2.130162,2.032316,1.920467,2.410054,2.66535,1.877020,2.033393,2.085099
std,0.632705,0.839422,0.737340,1.067143,0.605214,0.995449,0.778344,0.652131,1.229364,0.913004,1.044680,1.154553,1.06767,0.967673,0.827541,0.949210
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.00000,1.000000,1.000000,1.000000
50%,1.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000,2.000000,3.00000,2.000000,2.000000,2.000000
75%,1.000000,2.000000,2.000000,3.000000,1.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.000000,3.000000,4.00000,2.000000,3.000000,3.000000
max,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.00000,4.000000,4.000000,4.000000


# Use Lambda function to relabel Likert scale data



In [35]:
def likert_converter (column_name):
    """Replace Likert Scale numeric values with words"""
    if 'Q2_' in column_name:
        subset_df[column_name] = subset_df[column_name].map(lambda x: 'Very Important' if x==1 
        else ('Somewhat important' if x==2 
        else('Not so important'if x==3 
        else 'Not at all important')))
    else:
        subset_df[column_name] = subset_df[column_name].map(lambda x: 'Strongly agree' if x==1 
        else ('Somewhat agree' 
        if x==2 else('Somewhat disagree'if x==3 
        else 'Strongly disagree')))


#Replace with correct column list
for column in question_columns: 
    likert_converter(column)

 


In [36]:
subset_df

,Q2_1,Q2_2,Q2_3,Q2_4,Q2_5,Q2_6,Q2_7,Q2_8,Q2_9,Q2_10,...,Q3_3,Q3_4,Q3_5,Q3_6,educ,race,gender,income_cat,voter_category,age_binned
0,Very Important,Very Important,Somewhat important,Not at all important,Very Important,Not at all important,Somewhat important,Somewhat important,Not at all important,Somewhat important,...,Strongly disagree,Strongly disagree,Somewhat disagree,Somewhat agree,College,White,Female,$75-125k,always,66-94
1,Very Important,Somewhat important,Somewhat important,Not so important,Very Important,Very Important,Somewhat important,Very Important,Very Important,Not so important,...,Strongly disagree,Somewhat disagree,Somewhat disagree,Somewhat agree,College,White,Female,$125k or more,always,66-94
2,Very Important,Very Important,Somewhat important,Somewhat important,Very Important,Very Important,Somewhat important,Very Important,Not at all important,Not so important,...,Somewhat disagree,Somewhat disagree,Somewhat agree,Somewhat agree,College,White,Male,$125k or more,sporadic,37-54
3,Very Important,Very Important,Very Important,Not so important,Very Important,Very Important,Very Important,Very Important,Very Important,Somewhat important,...,Strongly disagree,Strongly disagree,Somewhat agree,Strongly agree,Some college,Black,Female,$40-75k,sporadic,55-65
6,Very Important,Very Important,Somewhat important,Not so important,Very Important,Very Important,Very Important,Very Important,Very Important,Not so important,...,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat agree,High school or less,White,Female,$125k or more,always,66-94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5831,Very Important,Not so important,Very Important,Not so important,Very Important,Somewhat important,Somewhat important,Somewhat important,Not so important,Very Important,...,Somewhat disagree,Strongly agree,Strongly agree,Somewhat agree,Some college,Hispanic,Male,$40-75k,always,22-36
5832,Very Important,Very Important,Very Important,Somewhat important,Very Important,Somewhat important,Very Important,Very Important,Very Important,Very Important,...,Somewhat disagree,Somewhat agree,Somewhat agree,Strongly agree,High school or less,White,Female,$125k or more,rarely/never,55-65
5833,Very Important,Somewhat important,Very Important,Not so important,Very Important,Very Important,Somewhat important,Very Important,Not at all important,Very Important,...,Strongly disagree,Somewhat agree,Strongly disagree,Somewhat agree,College,Other/Mixed,Male,$125k or more,sporadic,37-54
5834,Somewhat important,Somewhat important,Somewhat important,Somewhat important,Somewhat important,Somewhat important,Somewhat important,Somewhat important,Somewhat important,Somewhat important,...,Somewhat disagree,Somewhat disagree,Somewhat disagree,Somewhat disagree,High school or less,Black,Female,Less than $40k,always,22-36


## Rename columns


In [37]:
# Create dictionary to rename columns

column_dict = {"Q2_1":"Voting",
        "Q2_2": "Jury",
        "Q2_3": "Govt_Politics",
        "Q2_4": "American_Flag",
        "Q2_5": "US_Census",
        "Q2_6": "Pledge",
        "Q2_7":"Military Support",
        "Q2_8": "Respecting_Others",
        "Q2_9":"Believing in God",
        "Q2_10":"Protesting", 
        "Q3_1": "Racism_US",
        "Q3_2": "Racism_Policing",
        "Q3_3":"Society_Whole",
        "Q3_4":"Mainstream_Media",
        "Q3_5":"Traditional_Parties",
        "Q3_5":"Way_People_Talk"}


# Use rename method to update dictionary keys to values

subset_df = subset_df.rename(columns=column_dict)



In [43]:
# Check for missing values for our dataframe
subset_df.isna().sum()

Voting               0
Jury                 0
Govt_Politics        0
American_Flag        0
US_Census            0
Pledge               0
Military Support     0
Respecting_Others    0
Believing in God     0
Protesting           0
Racism_US            0
Racism_Policing      0
Society_Whole        0
Mainstream_Media     0
Way_People_Talk      0
Q3_6                 0
educ                 0
race                 0
gender               0
income_cat           0
voter_category       0
age_binned           0
dtype: int64

## Export df into csv file

In [39]:
subset_df.to_csv('american_values.csv')

# Perform exploratory analysis & visualization using Matplotlib and Seaborn

In [40]:
# Create visualizations for dataset 

# Ask and Answer questions about the data in a Jupyter Notebook

- Do people of certain backgrounds tend to think differently American values?
- Do certain answers correlate with each other?

# Summarize your inference and write a conclusion
